In [30]:
#Import the PuLP library.
from pulp import *

# Create an LP minimization problem named "Project planning" using LpProblem
model = LpProblem("Project planning", LpMinimize)


In [31]:
# Define a dictionary named workers that contains the names of different types of workers and their hourly rates.
workers = {
    'ProjectManager': 72,  # Example hourly rate for the project manager
    'FrontendDeveloper': 60,
    'BackendDeveloper': 60,
    'DataScientist': 58,
    'DataEngineer': 72,
}

In [32]:
# Define a dictionary named activities that contains the names of project tasks and their estimated durations (in hours) for the "BestCaseTime" scenario.
activities = {
    'Describe product': {'BestCaseTime': 2},
    'Develop marketing strategy': {'BestCaseTime': 8},
    'Design brochure': {'BestCaseTime': 6},
    #'Develop product protoype': {'BestCaseTime': 40},
    'Requirements analysis': {'BestCaseTime': 6},
    'Software design': {'BestCaseTime': 8},
    'System design':{'BestCaseTime': 8}, 
    'Coding': {'BestCaseTime': 20},
    'Write documentation': {'BestCaseTime': 10},
    'Unit testing': {'BestCaseTime': 10},
    'System testing': {'BestCaseTime': 12},
    'Package deliverables': {'BestCaseTime': 4},
    'Survey potential market': {'BestCaseTime': 10},
    'Develop pricing plan': {'BestCaseTime': 6},
    'Develop implementation  plan': {'BestCaseTime': 8},
    'Write client proposal': {'BestCaseTime': 4}

}


In [33]:
# Create a list named activities_list containing the names of all project tasks.
activities_list = list(activities.keys())


In [34]:
#Define a dictionary named precedences that specifies the precedence relationships between tasks. Tasks are organized hierarchically, with some tasks depending on others.
precedences = {
            'Describe product': [],
            'Develop marketing strategy': [], 
            'Design brochure': ['Describe product'], 
            #'Develop product protoype': [],  
            'Requirements analysis': ['Describe product'], 
            'Software design': ['Requirements analysis'], 
            'System design': ['Requirements analysis'], 
            'Coding': ['Software design','System design'],
            'Write documentation': ['Coding'],
            'Unit testing': ['Coding'],
            'System testing' : ['Unit testing'],
            'Package deliverables': ['Write documentation','System testing'],
            'Survey potential market':['Develop marketing strategy','Design brochure'],
            'Develop pricing plan': ['Package deliverables','Survey potential market'],
            'Develop implementation  plan': ['Describe product','Package deliverables'],
            'Write client proposal': ['Develop pricing plan','Develop implementation  plan']
            }


In [35]:
# Define decision variables x using LpVariable.dicts. These variables represent the number of hours each worker spends on each task. The decision variables are created for all worker-task combinations and have a lower bound of 0.
x = pulp.LpVariable.dicts("Hours", [(worker, task) for worker in workers for task in activities],
                           lowBound=0)


In [36]:
#Define the objective function using model += pulp.lpSum(...). The objective is to minimize the total project duration (in hours) for the "BestCaseTime" scenario. It is calculated as the sum of the product of worker hours and task durations.
model += pulp.lpSum(x[(worker, task)] * activities[task]['BestCaseTime'] for worker in workers for task in activities)

In [37]:
#Constraints
#Define the objective function using model += pulp.lpSum(...). The objective is to minimize the total project duration (in hours) for the "BestCaseTime" scenario. It is calculated as the sum of the product of worker hours and task durations.
for task in activities:
    model += pulp.lpSum(x[(worker, task)] for worker in workers) >= activities[task]['BestCaseTime']

# Ensure that workers are available
# Example: Assuming 160 hours of work per worker
#for worker in workers:
 #   model += pulp.lpSum(x[(worker, task)] for task in activities) <= 160


In [38]:
# Solve the linear programming problem
model.solve()

# Print the results
for worker in workers:
    for task in activities:
        if x[(worker, task)].varValue > 0:
            print(f"{worker} works {x[(worker, task)].varValue} hours on {task}")

print(f"Total BestCaseTime: {pulp.value(model.objective)}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/431e58f182d2443da1065ec79f829738-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/431e58f182d2443da1065ec79f829738-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 171 RHS
At line 187 BOUNDS
At line 188 ENDATA
Problem MODEL has 15 rows, 75 columns and 75 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-15) rows, 0 (-75) columns and 0 (-75) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1244
After Postsolve, objective 1244, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1244 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions chang

In [14]:
# Create the LP problem
prob = LpProblem("Critical Path", LpMinimize)


In [15]:
# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}


In [16]:
# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"


In [17]:
# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} days in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/4005328fed6747969c64b3d5f2095c9e-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/4005328fed6747969c64b3d5f2095c9e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 127 RHS
At line 163 BOUNDS
At line 164 ENDATA
Problem MODEL has 35 rows, 32 columns and 70 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-35) rows, 0 (-32) columns and 0 (-70) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 576
After Postsolve, objective 576, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 576 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed 